In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
from os import listdir
from os.path import isfile, join

In [ ]:
primary_files = [f for f in listdir('Lephare/lephare/test/PRIMARY_SOLS_DET277W/') if isfile(join('Lephare/lephare/test/PRIMARY_SOLS_DET277W/', f))]
#making list of file index 
for i in range(len(primary_files)):
    print(join(primary_files[i][6]+primary_files[i][7]+primary_files[i][8]+primary_files[i][9]+primary_files[i][10]))

In [ ]:
index = '19171' #manually change index as parameters need to be adjusted for each SED produced, so did not automate 
                #always has to be 5 digits - 2077->02077

In [ ]:
#looking for the primary, high redshift solution file
high = open('Lephare/lephare/test/PRIMARY_SOLS_DET277W/Id0000'+ str(index)+'.spec', 'r')
wavelength =[]
mag=[]
for _ in range(421): #skips the first 420 lines
    next(high)
for line in high:
    tok = line.split('  ')
    wavelength.append(float(tok[0]))
    mag.append(float(tok[1]))
high.close()


In [ ]:
#looking for the secondary, low redshift solution file - done separately to high redshift due to problem using lephare
low= open('Lephare/lephare/test/Id0000'+ str(index)+'.spec', 'r')
wavelength_low=[]
mag_low=[]

for _ in range(421):
    next(low)
for line in low:
    tok = line.split('  ')
    wavelength_low.append(float(tok[0]))
    mag_low.append(float(tok[1]))

low.close()

#get low redshift value from file
low= open('Lephare/lephare/test/Id0000'+ str(index)+'.spec', 'r')
content =low.readlines()    
for line in content[1]:
    test = re.sub("\s+", ",", content[1].strip())
    tokens = test.split(',')
zlow = np.round(float(tokens[2]),2)
low.close()
print(zlow)

In [ ]:
#get data points for each filter from primary solution file (as this is deemed to be best fit by lephare)

file = open('Lephare/lephare/test/PRIMARY_SOLS_DET277W/Id0000'+ str(index)+'.spec', 'r')
file_data = open('sed_plots/data_files/index'+str(index)+'_data_sed','w')
content =file.readlines()
for i in range(13,20):
    file_data.write(content[i])
file.close()
file_data.close()



#get high redshift value from file
for line in content[1]:
    test = re.sub("\s+", ",", content[1].strip())
    tokens = test.split(',')
zhigh = np.round(float(tokens[2]),2)
    
print(zhigh)

In [ ]:
file_data = open('sed_plots/data_files/index'+str(index)+'_data_sed','r')
wave_data=[]
mag_data=[]
mag_err=[]
wave_width=[]
for line in file_data:
    test = re.sub("\s+", ",", line.strip())
    tokens = test.split(',')
    wave_data.append(float(tokens[2]))
    mag_data.append(float(tokens[0]))
    mag_err.append(float(tokens[1]))
    wave_width.append(float(tokens[3])/2) #divide by two as wavelength width is not error both directions
#use fake mag to adjust positons of datapoints in filters f090w and f115w - often very faint so need to be increased to see
#on same scale as rest of graph
fake_mag = [mag_data[0]-1,mag_data[1]-1]


In [ ]:
#plotting the SED

plt.plot(wavelength, mag, color = 'darkmagenta',label = 'z='+str(zhigh) ) #high z template
plt.plot(wavelength_low, mag_low,color='sandybrown',  label ='z='+str(zlow)) #low z template

#plots from F150W to F444W as points with error bars
plt.errorbar(wave_data[2:7],mag_data[2:7],yerr = mag_err[2:7], xerr = wave_width[2:7],capsize = 2,fmt='o' ,color='k',zorder=6)

#plots points as lines for F090W and F115W. zorder brings these to front i.e infront of template lines
plt.errorbar(wave_data[0],fake_mag[0], xerr = wave_width[0],capsize = 2,fmt='-' ,color='k',zorder=6)
plt.errorbar(wave_data[1],fake_mag[1], xerr = wave_width[1],capsize = 2,fmt='-' ,color='k',zorder=6)


#makes arrow point downward to indicate these points are fainter than the rest of the points
plt.arrow(wave_data[0],fake_mag[0], dx = 0,dy=0.2,length_includes_head=True,
  head_width=1000, head_length=0.05,color='k',zorder=6)
plt.arrow(wave_data[1],fake_mag[1], dx = 0,dy=0.2,length_includes_head=True,
       head_width=1000, head_length=0.05,color='k',zorder=6)

plt.gca().invert_yaxis()

plt.ylim(32,27) #axis limits adapted from Castellano 2022
plt.xlim(0,5e4)
plt.legend()
plt.title('Candidate '+str(index))
plt.xlabel('Wavelength ($\AA$)')
plt.ylabel('Magnitudes')
plt.savefig('sed_plots/det277w/Candidate_'+str(index)+'.png')

In [ ]:
#PLOTTING CHI SQ FROM LEPHARE OUTPUT
k= np.linspace(0,20,401)  #redshift steps
chiindex='19171'
chifile = open('Lephare/lephare/test/Id0000'+ str(chiindex)+'.chi', 'r') #lephare output

z =[]
chi=[]
chimin =[]
for _ in range(1):
    next(chifile)
for line in chifile:
    tok = line.split('  ')
    z.append(float(tok[0]))
    chi.append(float(tok[11]))

chifile.close()

file=open('sed_plots/chi_min'+str(chiindex),'a')
for i in range(len(z)):
    for j in range(len(k)):
        chi_sort=[]
        if k[j]<z[i]<k[j+1]: # for every redshift value in redshift bin of 0.05
            
            chi_sort.append(chi[i]) #takes every chi sq value and finds the minimum
            chimin = np.min(chi_sort)
            file.write(str(chimin)+','+str(z[i])+'\n') #read out to file to be plotted
file.close()
        


In [ ]:
file=open('sed_plots/chi_min_test_files/18534_chi_min.txt','r')
chi_file=[]
z_file=[]
for line in file:
    tok = line.split(',')
    chi_file.append(float(tok[0]))
    z_file.append(float(tok[1]))
file.close()

In [ ]:
plt.plot(z_file,chi_file,linewidth =2.5, color = 'k')
plt.ylim(0,25)    #want chi sq to be less than 25 for good candidates, this helps see the minimum clearly 
plt.ylabel('$\u03C7^2$',fontsize=18)
plt.xlabel('z',fontsize=18)
plt.title('Candidate '+str(chiindex))
plt.savefig('sed_plots/chi_sq_plots/chi_sq_'+str(chiindex)+'.png')